In [1]:
import numpy as np
import random
from tensorflow.keras import *
from tensorflow.keras.layers import *
#from generations_comnet import *
import tensorflow as tf
import matplotlib.pyplot as plt
from keras import backend as backend
import os
import scipy.io as sio
import argparse
from scipy.interpolate import interp1d



# Completed code

## load data and communication simulation

In [8]:
### import channel matrix for training and testing
num_train = 500*2500 # 2500 PDP, each PDP 500 channel,
num_test = 10000 * 5 # 5 PDP, each PDP 10000 channel


K = 72#length of symbol after modulation


#channel_train = np.random.rand(num_train, K) + 1j*np.random.rand(num_train, K)
#channel_test = np.random.rand(num_test, K) + 1j*np.random.rand(num_test, K)
channel_train = np.loadtxt('path_for_channel_train.txt')
channel_test = np.loadtxt('path_for_channel_test.txt')
Pilot_file_name = 'file_path.txt'

####################### Training configuration #######################
num_channel_per_PDP = 500
num_training_per_batch = 500
steps_per_epoch_train = 1
epochs_train = 1
saving_path_cross_attention = 'save_path_cross_attention.h5'


####################### Testing configuration #######################
num_testing_PDP = 5
num_testing_channel_per_PDP = 500
SNR_list = [5,10,15,20,25]

####################### function for conventional wireless communication #######################
num_few_shots = 16 #### number of few-shot samples during new environment adaptation
feature_dims = 32 #### feature map dimension
CP = K
P = 18#num of pilots
allCarriers = np.arange(K)  # indices of all subcarriers position([0, 1, ... K-1])
mu = 2
payloadBits_per_OFDM = K * mu
bitses = np.random.binomial(n=1, p=0.5, size=(num_testing_channel_per_PDP*mu, payloadBits_per_OFDM)) ### save it in the first time to keep testing bits same
#bitses = np.loadtxt('path_for_testing_bits_72.txt',delimiter=',')
SNRdb = 25
if P < K:
    pilotCarriers = allCarriers[::K // P]  # Pilots is every (K/P)th carrier.
    #pilotCarriers[-1] = 71
    dataCarriers = np.delete(allCarriers, pilotCarriers)

else:  # K = P
    pilotCarriers = allCarriers
    dataCarriers = []

def interp(y):

    x = np.arange(len(y))
    idx = np.nonzero(y)
    interp = interp1d(x[idx],y[idx])

    return interp(x)

def Modulation(bits):
    bit_r = bits.reshape((int(len(bits) / mu), mu))
    # This is just for QPSK modulation
    return np.sqrt(2)*(2 * bit_r[:, 0] - 1) + np.sqrt(2)*1j * (2 * bit_r[:, 1] - 1)

def OFDM_symbol(Data, pilot_flag):
    symbol = np.zeros(K, dtype=complex)  # the overall K subcarriers
    #symbol = np.zeros(K)
    symbol[pilotCarriers] = pilotValue  # allocate the pilot subcarriers
    return symbol


def IDFT(OFDM_data):
    return np.fft.ifft(OFDM_data)


def addCP(OFDM_time):
    #cp = OFDM_time[-CP:]
    cp = OFDM_time[0:CP]             # take the last CP samples ...
    #return np.hstack([cp, OFDM_time])  # ... and add them to the beginning
    return np.concatenate((cp, OFDM_time))


def channel(signal, channelResponse, SNRdb):
    convolved = np.convolve(signal, channelResponse)
    signal_power = np.mean(abs(convolved**2))
    sigma2 = signal_power * 10**(-SNRdb / 10)
    noise = np.sqrt(sigma2 / 2) * (np.random.randn(*
                                                   convolved.shape) + 1j * np.random.randn(*convolved.shape))
    return convolved + noise
    #return convolved


def removeCP(signal):
    return signal[CP:(CP + K)]


def DFT(OFDM_RX):
    return np.fft.fft(OFDM_RX)


def ofdm_simulate(codeword, channelResponse, SNRdb):
    """
    :param codeword: data to modulation
    """
    # ----- Calculate h_ls using pilots   ---
    bits = np.random.binomial(n=1, p=0.5, size=(2*(K - P),))
    QAM = Modulation(bits)
    OFDM_data = np.zeros(K, dtype=complex)
    h_ls = np.zeros(K, dtype=complex)
    OFDM_data[pilotCarriers] = pilotValue#put pilot at position [0:64]
    OFDM_data[dataCarriers] = QAM
    OFDM_time = IDFT(OFDM_data)
    OFDM_withCP = addCP(OFDM_time)
    OFDM_TX = OFDM_withCP
    OFDM_RX = channel(OFDM_TX, channelResponse, SNRdb)
    OFDM_RX_noCP = removeCP(OFDM_RX)
    OFDM_RX_noCP = DFT(OFDM_RX_noCP)
    h_ls[pilotCarriers]=OFDM_RX_noCP[pilotCarriers]/pilotValue#LS estimation
    h_ls_concat = np.concatenate((np.real(h_ls), np.imag(h_ls)))
    h_ls_concat = h_ls_concat.reshape(2,K)
    h_ls_reshape = h_ls_concat.T





    h_true = np.fft.fft(channelResponse)
    h_concat = np.concatenate((np.real(h_true), np.imag(h_true)))
    h_concat = h_concat.reshape(2,K)
    h_groundtruth = h_concat.T


    # ----- target inputs ---
    symbol = np.zeros(K, dtype=complex)
    codeword_qam = Modulation(codeword)#modualte 128 data bits data into 64 symbol
    symbol[np.arange(K)] = codeword_qam
    OFDM_data_codeword = symbol
    OFDM_time_codeword = np.fft.ifft(OFDM_data_codeword)
    OFDM_withCP_cordword = addCP(OFDM_time_codeword)
    OFDM_RX_codeword = channel(OFDM_withCP_cordword, channelResponse, SNRdb)
    OFDM_RX_noCP_codeword = removeCP(OFDM_RX_codeword)
    OFDM_RX_noCP_codeword = DFT(OFDM_RX_noCP_codeword)
    return (h_ls_reshape,
            np.concatenate((np.real(OFDM_RX_noCP_codeword), np.imag(OFDM_RX_noCP_codeword)))
            , h_groundtruth)


if os.path.isfile(Pilot_file_name):
    print('Load Training Pilots txt')
    # load file
    bits = np.loadtxt(Pilot_file_name, delimiter=',')
else:
    # write file
    bits = np.random.binomial(n=1, p=0.5, size=(P * mu, ))#randomly generate data via binomal distribution
    np.savetxt(Pilot_file_name, bits, delimiter=',')


pilotValue = Modulation(bits)#modulate data into pilots

Load Training Pilots txt


## generate dataset for the meta-learner

In [9]:
bits_pilot_few_shot = np.random.choice([0, 1], size=K*mu)
pilotValue_few_shot = Modulation(bits_pilot_few_shot)
pilotCarriers_few_shot = np.arange(K)



def ofdm_simulate_hypernet_train(channelResponse, SNRdb):
    """
    :param codeword: data to modulation
    """
    # ----- Calculate h_ls using pilots   ---
    OFDM_data = np.zeros(K, dtype=complex)
    h_ls = np.zeros(K, dtype=complex)

    OFDM_data[pilotCarriers_few_shot] = pilotValue_few_shot#put pilot at position [0:64]
    OFDM_time = IDFT(OFDM_data)
    OFDM_withCP = addCP(OFDM_time)
    OFDM_TX = OFDM_withCP
    OFDM_RX = channel(OFDM_TX, channelResponse, SNRdb)
    OFDM_RX_noCP = removeCP(OFDM_RX)
    OFDM_RX_noCP = DFT(OFDM_RX_noCP)
    #h_ls = OFDM_RX_noCP/pilotValue#LS estimation
    #h_true = np.fft.fft(channelResponse)
    h_ls[pilotCarriers_few_shot]=OFDM_RX_noCP[pilotCarriers_few_shot]/pilotValue_few_shot#LS estimation
    h_ls_concat = np.concatenate((np.real(h_ls), np.imag(h_ls)))
    h_ls_concat = h_ls_concat.reshape(2,K)
    h_ls_reshape = h_ls_concat.T


    h_true = np.fft.fft(channelResponse)
    h_concat = np.concatenate((np.real(h_true), np.imag(h_true)))
    h_concat = h_concat.reshape(2,K)
    h_groundtruth = h_concat.T

    return h_ls_reshape, h_groundtruth


def hyper_set_generation():
    h_hyper_train_dataset = []
    h_hyper_train_label = []
    h_hyper_test_dataset = []
    h_hyper_test_label = []

    for i in range(len(channel_train)):
      H = channel_train[i]
      h_ls,h_true = ofdm_simulate_hypernet_train(H, SNRdb)
      #h_ls_time_domain = np.fft.ifft(h_ls)
      h_hyper_train_dataset.append(h_ls)
      h_hyper_train_label.append(h_true)


    h_hyper_train_dataset = np.array(h_hyper_train_dataset)
    h_hyper_train_label = np.array(h_hyper_train_label)


    for i in range(len(channel_test)):
      H = channel_test[i]
      h_ls,h_true = ofdm_simulate_hypernet_train(H, SNRdb)
      #h_ls_time_domain = np.fft.ifft(h_ls)
      h_hyper_test_dataset.append(h_ls)
      h_hyper_test_label.append(h_true)

    h_hyper_test_dataset = np.array(h_hyper_test_dataset)
    h_hyper_test_label = np.array(h_hyper_test_label)
    return h_hyper_train_dataset, h_hyper_train_label, h_hyper_test_dataset, h_hyper_test_label

def hyper_set_generation_various_SNR():
    h_hyper_test_dataset_various_SNR = []
    SNR_list = [5,10,15,20,25]
    for i in range(5):
      SNRdb = SNR_list[i]
      h_hyper_test_dataset_copy = []
      for i in range(len(channel_test)):
        H = channel_test[i]
        h_ls,h_true = ofdm_simulate_hypernet_train(H, SNRdb)
        #h_ls_time_domain = np.fft.ifft(h_ls)
        h_hyper_test_dataset_copy.append(h_ls)
      h_hyper_test_dataset_various_SNR.append(np.array(h_hyper_test_dataset_copy))

    h_hyper_test_dataset_various_SNR = np.array(h_hyper_test_dataset_various_SNR)
    return h_hyper_test_dataset_various_SNR

h_hyper_train_dataset, h_hyper_train_label, h_hyper_test_dataset, h_hyper_test_label = hyper_set_generation()
h_hyper_test_dataset_various_SNR = hyper_set_generation_various_SNR()

## pre-train feature extraction network

In [ ]:
def feature_extractor(feature_dims):
  input_LS = Input(shape=(int(payloadBits_per_OFDM/2),2),name = 'input')  # Channel LS estimation

  x_1 = Conv1D(64*5, 8, padding = 'same', activation = 'linear',name = 'extractor_1')(input_LS)
  x_2 = Conv1D(64*5, 8, padding = 'same', activation = 'linear',name = 'extractor_2')(x_1)
  x_3 = Conv1D(feature_dims, 3, padding = 'same', activation = 'linear',name = 'extractor_3')(x_2)

  final_output = Conv1D(2, 2,padding = 'same',activation = 'linear',name = 'channel6')(x_3)

  model = Model(inputs=input_LS, outputs=final_output)
  return model


def pre_trained_feature_extractor(feature_dims):
  input_LS = Input(shape=(int(payloadBits_per_OFDM/2),2),name = 'input')  # Channel LS estimation

  x_1 = Conv1D(64*5, 8, padding = 'same', activation = 'linear',name = 'extractor_1')(input_LS)
  x_2 = Conv1D(64*5, 8, padding = 'same', activation = 'linear',name = 'extractor_2')(x_1)
  x_3 = Conv1D(feature_dims, 3, padding = 'same', activation = 'linear',name = 'extractor_3')(x_2)

  model = Model(inputs=input_LS, outputs=x_3)
  return model

def generate_hyper_samples(class_idx, num_combined, h_hyper_train_dataset, h_hyper_train_label, SNR_ratio):
  idx = random.sample(range(class_idx*num_channel_per_PDP,(class_idx+1)*num_channel_per_PDP), num_combined)
  return h_hyper_train_dataset[idx],h_hyper_train_label[idx]

def training_gen_hyper(bs, num_combined, h_hyper_train_dataset, h_hyper_train_label, SNRdb = 20):
    while True:
        index = np.random.choice(np.arange(len(channel_train)), size=bs)#choose bs daa from trainsets
        H_total = channel_train[index]
        input_hls = []
        input_h_labels = []

        #for H in H_total:
        for i in index:
            #H = channel_trainingset[index]
            H =channel_train[i]
            class_idx = int(i/num_channel_per_PDP)
            vec_input, vec_true= generate_hyper_samples(class_idx,num_combined, h_hyper_train_dataset, h_hyper_train_label, SNRdb)
            input_hls.append(vec_input)
            input_h_labels.append(vec_true)


        yield ({'input':np.asarray(input_hls)}, {'channel6':np.asarray(input_h_labels)})
        #return np.asarray(input_vec)

def validation_gen_hyper(bs, num_combined, h_hyper_train_dataset, h_hyper_train_label, SNRdb = 20):
    while True:
        index = np.random.choice(np.arange(len(channel_train)), size=bs)
        H_total = channel_train[index]

        input_hls = []
        input_h_labels = []
        for i in index:
            H=channel_train[i]
            class_idx = int(i/num_channel_per_PDP)
            vec_input, vec_true= generate_hyper_samples(class_idx,num_combined,h_hyper_train_dataset, h_hyper_train_label, SNRdb)
            input_hls.append(vec_input)
            input_h_labels.append(vec_true)

        # sio.savemat('y_real+imag_{}.mat'.format(str(SNRdb)),{'y_SNR_{}'.format(str(SNRdb)):input_data})
        # sio.savemat('init_bits_{}.mat'.format(str(SNRdb)), {'bit_SNR_{}'.format(str(SNRdb)):all_bits})
        # sio.savemat('h_ls_real+imag_{}.mat'.format(str(SNRdb)), {'h_SNR_{}'.format(str(SNRdb)): input_hls})

        yield ({'input':np.asarray(input_hls)}, {'channel6':np.asarray(input_h_labels)})

def train_for_feature_extraction(num_few_shots, feature_dims, h_hyper_train_dataset, h_hyper_train_label):
  num_combined = num_few_shots
  model_extractor = feature_extractor(feature_dims)

  model_extractor.compile(optimizer='adam', loss='mse',  metrics=[tf.keras.metrics.MeanSquaredError()])
  checkpoint = callbacks.ModelCheckpoint('dual.h5',
                                          verbose=0, save_best_only=True, mode='min', save_weights_only=True)
  model_extractor.fit_generator(
      training_gen_hyper(num_training_per_batch,num_combined, h_hyper_train_dataset, h_hyper_train_label, SNRdb),
      steps_per_epoch=steps_per_epoch_train,
      epochs=epochs_train,
      validation_data=validation_gen_hyper(num_training_per_batch,num_combined,h_hyper_train_dataset, h_hyper_train_label, SNRdb),
      validation_steps=1,
      callbacks=[checkpoint],
      verbose=2)
  return model_extractor





def feature_map_extraction(model, feature_dims, h_hyper_train_dataset):
  feature_extractor_pre_trained = pre_trained_feature_extractor(feature_dims)

  feature_extractor_pre_trained.compile(optimizer='adam', loss='mse', metrics=[tf.keras.metrics.MeanSquaredError()])

  layername = ['extractor_1','extractor_2','extractor_3']
  for i in range(len(layername)):
    for layer in feature_extractor_pre_trained.layers:
      if layer.name == layername[i]:
          layer.trainable = False

  for i in range(len(layername)):
    for layer in feature_extractor_pre_trained.layers:
      if layer.name == layername[i]:
        for layerss in model.layers:
          if layerss.name == layername[i]:
            layer.set_weights(layerss.get_weights())


  h_hyper_feature_map = []
  for i in range(int(len(h_hyper_train_dataset)/1000)):
    input_data = h_hyper_train_dataset[int(i*1000):int((i+1)*1000)]
    input_feature_map = feature_extractor_pre_trained.predict(input_data, verbose=0)
    h_hyper_feature_map.append(input_feature_map)

  h_hyper_feature_map = np.array(h_hyper_feature_map)
  h_hyper_feature_map = h_hyper_feature_map.reshape(len(h_hyper_train_dataset),K,feature_dims)
  return h_hyper_feature_map


def feature_map_extraction_varying_SNR(model, feature_dims, h_hyper_test_dataset_various_SNR):
  feature_extractor_pre_trained = pre_trained_feature_extractor(feature_dims)

  feature_extractor_pre_trained.compile(optimizer='adam', loss='mse', metrics=[tf.keras.metrics.MeanSquaredError()])

  layername = ['extractor_1','extractor_2','extractor_3']
  for i in range(len(layername)):
    for layer in feature_extractor_pre_trained.layers:
      if layer.name == layername[i]:
          layer.trainable = False

  for i in range(len(layername)):
    for layer in feature_extractor_pre_trained.layers:
      if layer.name == layername[i]:
        for layerss in model.layers:
          if layerss.name == layername[i]:
            layer.set_weights(layerss.get_weights())


  SNR_list = [5,10,15,20,25]
  h_hyper_testing_feature_map_various_SNR = []
  for z in range(5):
    h_hyper_testing_feature_map = []
    for i in range(int(len(h_hyper_test_dataset_various_SNR[0])/1000)):
      input_data = h_hyper_test_dataset_various_SNR[z,int(i*1000):int((i+1)*1000)]
      input_feature_map = feature_extractor_pre_trained.predict(input_data, verbose=0)
      h_hyper_testing_feature_map.append(input_feature_map)

    h_hyper_testing_feature_map = np.array(h_hyper_testing_feature_map)
    h_hyper_testing_feature_map = h_hyper_testing_feature_map.reshape(50000,72,32)
    h_hyper_testing_feature_map_various_SNR.append(h_hyper_testing_feature_map)

  h_hyper_testing_feature_map_various_SNR = np.array(h_hyper_testing_feature_map_various_SNR)
  return h_hyper_testing_feature_map_various_SNR



feature_extraction = train_for_feature_extraction(num_few_shots, feature_dims, h_hyper_train_dataset, h_hyper_train_label)
h_hyper_feature_map = feature_map_extraction(feature_extraction, feature_dims, h_hyper_train_dataset)
h_hyper_testing_feature_map_various_SNR = feature_map_extraction_varying_SNR(feature_extraction, feature_dims, h_hyper_test_dataset_various_SNR)

<ipython-input-11-81ee5601d85a>:75: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model_extractor.fit_generator(


1/1 - 23s - loss: 24.4415 - mean_squared_error: 24.4415 - val_loss: 1.8483 - val_mean_squared_error: 1.8483 - 23s/epoch - 23s/step


## cross_task_attention

In [ ]:
def generate_few_shot_samples(class_idx, num_few_shots, h_hyper_train_dataset, h_hyper_feature_map):
  idx = random.sample(range(class_idx*num_channel_per_PDP,(class_idx+1)*num_channel_per_PDP), num_few_shots)
  return h_hyper_train_dataset[idx], h_hyper_feature_map[idx]

def training_gen(bs, num_few_shots, h_hyper_train_dataset, h_hyper_feature_map, train_size, SNRdb = 20):
    while True:
        index = np.random.choice(np.arange(train_size), size=bs)#choose bs daa from trainsets
        H_total = channel_train[index]
        input_hls = []
        input_h_labels = []
        input_vec = []
        input_vec_part_feature_map = []
        input_h_ls_feature_map = []
        for i in index:
            H =channel_train[i]
            class_idx = int(i/num_channel_per_PDP)
            vec_part, vec_part_feature_map = generate_few_shot_samples(class_idx, num_few_shots, h_hyper_train_dataset, h_hyper_feature_map)
            input_vec_part_feature_map.append(vec_part_feature_map)

            input_vec.append(vec_part)

            s_bits = np.random.binomial(n=1, p=0.5, size=(payloadBits_per_OFDM,))#randomly generate s_bits with length 128 bits
            h_ls,signal_output, h_true = ofdm_simulate(s_bits, H, SNRdb)
            input_hls.append(h_ls)#add h_ls to input_1
            input_h_labels.append(h_true)

        yield ({'input_1':np.asarray(input_hls),'input_2':np.asarray(input_vec),'input_3':np.asarray(input_vec_part_feature_map)}, {'channel6':np.asarray(input_h_labels)})


def validation_gen(bs, num_few_shots, h_hyper_train_dataset, h_hyper_feature_map, train_size, SNRdb = 20):
    while True:
        index = np.random.choice(np.arange(train_size), size=bs)
        H_total = channel_train[index]
        input_hls = []
        input_h_labels = []
        input_vec = []
        input_vec_part_feature_map = []
        input_h_ls_feature_map = []
        for i in index:
            H =channel_train[i]
            class_idx = int(i/num_channel_per_PDP)
            vec_part,vec_part_feature_map = generate_few_shot_samples(class_idx, num_few_shots, h_hyper_train_dataset, h_hyper_feature_map)
            input_vec_part_feature_map.append(vec_part_feature_map)

            input_vec.append(vec_part)

            s_bits = np.random.binomial(n=1, p=0.5, size=(payloadBits_per_OFDM,))#randomly generate s_bits with length 128 bits
            h_ls,signal_output, h_true = ofdm_simulate(s_bits, H, SNRdb)

            input_hls.append(h_ls)#add h_ls to input_1
            input_h_labels.append(h_true)

        yield ({'input_1':np.asarray(input_hls),'input_2':np.asarray(input_vec),'input_3':np.asarray(input_vec_part_feature_map)}, {'channel6':np.asarray(input_h_labels)})

def meta_learning(num_few_shots, feature_dims):
  input_GAP = Input(shape=(num_few_shots,1,K), name = 'input')
  RP_att = Conv1D(16,1,padding = "same", activation = 'relu',name = 'meta_leaner_1_1')(input_GAP)
  RP_att = Conv1D(K,1,padding = "same", activation = 'linear',name = 'meta_leaner_1_2')(RP_att)

  model = Model(inputs=input_GAP, outputs=RP_att)
  return model

def model_cross_attention_CNN(net_meta_learner, num_few_shots, feature_dims):
  input_LS = Input(shape=(int(payloadBits_per_OFDM/2),2),name = 'input_1')  # Channel LS estimation
  input_vec = Input(shape=(num_few_shots,K, 2,),name = 'input_2')
  input_vec_feature_map = Input(shape=(num_few_shots,K, feature_dims,),name = 'input_3')
  #input_LS_feature_map = Input(shape=(1,K, feature_dims,),name = 'input_4')
  x_1 = Conv1D(64*5, 8, padding = 'same', activation = 'linear',name = 'extractor_1')(input_LS)
  x_2 = Conv1D(64*5, 8, padding = 'same', activation = 'linear',name = 'extractor_2')(x_1)
  input_LS_feature_map = Conv1D(feature_dims, 3, padding = 'same', activation = 'linear',name = 'extractor_3')(x_2)

  vec_feature_map_reshape = tf.reshape(input_vec_feature_map,[-1, input_vec_feature_map.shape[1]*input_vec_feature_map.shape[2],input_vec_feature_map.shape[3]])
  vec_feature_map_normalized = tf.linalg.normalize(vec_feature_map_reshape,axis = 2)[0]
  #vec_feature_map_normalized_reshape = tf.reshape(vec_feature_map_normalized,[-1,input_vec_feature_map.shape[1],input_vec_feature_map.shape[2],input_vec_feature_map.shape[3]])
  #LS_feature_map_reshape = tf.reshape(input_LS_feature_map,[-1, input_vec_feature_map.shape[2],input_vec_feature_map.shape[3]])
  LS_feature_map_reshape = input_LS_feature_map
  LS_feature_map_normalized = tf.linalg.normalize(LS_feature_map_reshape,axis = 2)[0]
  LS1_T = tf.transpose(LS_feature_map_normalized,perm = [0,2,1])
  R_combined = tf.matmul(vec_feature_map_normalized,LS1_T)
  R_P = tf.reshape(R_combined,[-1,num_few_shots, K,R_combined.shape[2]])
  R_Q = tf.transpose(R_P,perm = [0,1,3,2])
  RP_GAP = tf.reduce_mean(R_P, axis = 2, keepdims=True)
  RP_att = net_meta_learner(RP_GAP)
  RP_att = tf.transpose(RP_att,perm = [0,1,3,2])
  RP_multiply = tf.matmul(R_P,  RP_att)
  RP_attention = tf.nn.softmax(RP_multiply)+1.0
  weighted_vec_feature_map = input_vec_feature_map * RP_attention

  RQ_GAP = tf.reduce_mean(R_Q, axis = 2, keepdims=True)
  RQ_att = net_meta_learner(RQ_GAP)
  RQ_att = tf.transpose(RQ_att,perm = [0,1,3,2])
  RQ_multiply = tf.matmul(R_Q,  RQ_att)
  RQ_attention = tf.nn.softmax(RQ_multiply)+1.0
  LS_feature_map_reshape = tf.reshape(LS_feature_map_reshape,[-1,1,LS_feature_map_reshape.shape[1],LS_feature_map_reshape.shape[2]])
  LS_feature_map_reshape = tf.tile(LS_feature_map_reshape,[1,num_few_shots,1,1])
  weighted_LS_feature_map = LS_feature_map_reshape * RQ_attention

  feature_map = tf.concat([weighted_vec_feature_map,weighted_LS_feature_map],2)
  CNN_input_feature = tf.reshape(feature_map,[-1,num_few_shots*2,K,feature_dims])





  x1 = Conv2D(64*3, (2,1), strides = (2,1), activation = 'relu',name = 'conv1')(CNN_input_feature)
  #x1p = AveragePoolin2g2D(pool_size=(5, 1),strides=(5, 1), padding='valid')(x1)
  x2 = Conv2D(64*3, 5, padding = "same", activation = 'relu',name = 'conv2')(x1)
  #x2p = AveragePooling2D(pool_size=(2, 1),strides=(2, 1), padding='valid')(x2)
  x3 = Conv2D(64*3, 5, padding = "same", activation = 'relu',name = 'conv3')(x2)
  #x3p = AveragePooling2D(pool_size=(2, 1),strides=(2, 1), padding='valid')(x3)
  x3 = x1+x3
  x4 = Conv2D(64, (2,1), strides = (2,1), activation = 'relu',name = 'conv4')(x3)

  x5 = Conv2D(64, 5, padding = "same", activation = 'relu',name = 'conv5')(x4)

  x6 = Conv2D(64, 5, padding = "same", activation = 'relu',name = 'conv6')(x5)
  x6 = x6+x4

  x7 = Conv2D(8, 3, padding = "same", activation = 'linear',name = 'conv7')(x6)
  x8 = tf.reduce_mean(x7, axis = 1, keepdims=False)






  final_output_1 = Conv1D(2, 2,padding = 'same',activation = 'linear',name = 'channel6')(x8)
  model = Model(inputs=[input_LS, input_vec,input_vec_feature_map], outputs=final_output_1)
  return model


def model_cross_Task_CNN(net_meta_learner, num_few_shots, feature_dims):
  input_LS = Input(shape=(int(payloadBits_per_OFDM/2),2),name = 'input_1')  # Channel LS estimation
  input_vec = Input(shape=(num_few_shots,K, 2,),name = 'input_2')
  input_vec_feature_map = Input(shape=(num_few_shots,K, feature_dims,),name = 'input_3')







  ################ Task Attntion ################
  x1 = Conv2D(64, 8, padding = "same", activation = 'relu',name = 'conv11')(input_vec)
  x2 = Conv2D(32, 5, padding = "same", activation = 'relu',name = 'conv22')(x1)
  x3 = Conv2D(16, 5, padding = "same", activation = 'relu',name = 'conv33')(x2)
  x4 = Conv2D(4, 2, padding = "same", activation = 'relu',name = 'conv44')(x3)
  x5 = Flatten()(x4)
  x5 = Dense(128,activation = 'relu', use_bias=False,name = 'dense128')(x5)

  see1 = Dense(64*5, activation="sigmoid", use_bias=False,name = 'see1')(x5)

  see2 = Dense(64*5, activation="sigmoid", use_bias=False,name = 'see2')(x5)

  see3 = Dense(64*5, activation="sigmoid", use_bias=False,name = 'see3')(x5)









  ################ Cross Attntion ################
  x_1 = Conv1D(64*5, 8, padding = 'same', activation = 'linear',name = 'extractor_1')(input_LS)
  x_2 = Conv1D(64*5, 8, padding = 'same', activation = 'linear',name = 'extractor_2')(x_1)
  input_LS_feature_map = Conv1D(feature_dims, 3, padding = 'same', activation = 'linear',name = 'extractor_3')(x_2)

  vec_feature_map_reshape = tf.reshape(input_vec_feature_map,[-1, input_vec_feature_map.shape[1]*input_vec_feature_map.shape[2],input_vec_feature_map.shape[3]])
  vec_feature_map_normalized = tf.linalg.normalize(vec_feature_map_reshape,axis = 2)[0]
  LS_feature_map_reshape = input_LS_feature_map
  LS_feature_map_normalized = tf.linalg.normalize(LS_feature_map_reshape,axis = 2)[0]
  LS1_T = tf.transpose(LS_feature_map_normalized,perm = [0,2,1])
  R_combined = tf.matmul(vec_feature_map_normalized,LS1_T)
  R_P = tf.reshape(R_combined,[-1,num_few_shots, K,R_combined.shape[2]])
  R_Q = tf.transpose(R_P,perm = [0,1,3,2])
  RP_GAP = tf.reduce_mean(R_P, axis = 2, keepdims=True)
  RP_att = net_meta_learner(RP_GAP)
  RP_att = tf.transpose(RP_att,perm = [0,1,3,2])
  RP_multiply = tf.matmul(R_P,  RP_att)
  RP_attention = tf.nn.softmax(RP_multiply)+1.0
  weighted_vec_feature_map = input_vec_feature_map * RP_attention

  RQ_GAP = tf.reduce_mean(R_Q, axis = 2, keepdims=True)
  RQ_att = net_meta_learner(RQ_GAP)
  RQ_att = tf.transpose(RQ_att,perm = [0,1,3,2])
  RQ_multiply = tf.matmul(R_Q,  RQ_att)
  RQ_attention = tf.nn.softmax(RQ_multiply)+1.0
  LS_feature_map_reshape = tf.reshape(LS_feature_map_reshape,[-1,1,LS_feature_map_reshape.shape[1],LS_feature_map_reshape.shape[2]])
  LS_feature_map_reshape = tf.tile(LS_feature_map_reshape,[1,num_few_shots,1,1])
  weighted_LS_feature_map = LS_feature_map_reshape * RQ_attention

  feature_map = tf.concat([weighted_vec_feature_map,weighted_LS_feature_map],2)
  CNN_input_feature = tf.reshape(feature_map,[-1,num_few_shots*2,K,feature_dims])





  ################ Main Network ################
  x1 = Conv2D(64*3, (2,1), strides = (2,1), activation = 'relu',name = 'conv1')(CNN_input_feature)
  x2 = Conv2D(64*3, 5, padding = "same", activation = 'relu',name = 'conv2')(x1)
  x3 = Conv2D(64*3, 5, padding = "same", activation = 'relu',name = 'conv3')(x2)
  x3 = x1+x3
  x4 = Conv2D(64, (2,1), strides = (2,1), activation = 'relu',name = 'conv4')(x3)
  x5 = Conv2D(64, 5, padding = "same", activation = 'relu',name = 'conv5')(x4)
  x6 = Conv2D(64, 5, padding = "same", activation = 'relu',name = 'conv6')(x5)
  x6 = x6+x4
  x7 = Conv2D(8, 3, padding = "same", activation = 'linear',name = 'conv7')(x6)
  x8 = tf.reduce_mean(x7, axis = 1, keepdims=False)
  final_output_1 = Conv1D(2, 2,padding = 'same',activation = 'linear',name = 'channel6_1')(x8)
  x_1 = Conv1D(64*5, 8, padding = 'same', activation = 'linear',name = 'channel1')(final_output_1)
  sec = see1
  sec = tf.reshape(sec, [-1,1,64*5])
  x_1 = x_1*sec
  x = Conv1D(64*5, 8, padding = 'same', activation = 'linear', name = 'channel2')(x_1)
  sec = see2
  sec = tf.reshape(sec, [-1,1,64*5])
  x = x*sec
  x = x+x_1
  decoder_feature = Conv1D(64*5, 3, padding = 'same', activation = 'linear',name = 'channel3')(x)
  sec = see3
  sec = tf.reshape(sec, [-1,1,64*5])
  decoder_feature = decoder_feature*sec
  final_output = Conv1D(2, 2,padding = 'same',activation = 'linear',name = 'channel6')(decoder_feature)
  model = Model(inputs=[input_LS, input_vec,input_vec_feature_map], outputs=final_output)
  return model

In [ ]:
def run_experiment(model, num_few_shots, feature_dims, h_hyper_train_dataset, h_hyper_feature_map):
  net1 = meta_learning(num_few_shots, feature_dims)
  model_cross = model_cross_attention_CNN(net1, num_few_shots, feature_dims)
  model_cross.compile(optimizer='adam', loss='mse',  metrics=[tf.keras.metrics.MeanSquaredError()])
  checkpoint = callbacks.ModelCheckpoint('dual.h5',
                                          verbose=0, save_best_only=True, mode='min', save_weights_only=True)




  layername = ['extractor_1','extractor_2','extractor_3']
  for i in range(len(layername)):
    for layer in model_cross.layers:
      if layer.name == layername[i]:
          layer.trainable = False

  for i in range(len(layername)):
    for layer in model_cross.layers:
      if layer.name == layername[i]:
        for layerss in model.layers:
          if layerss.name == layername[i]:
            layer.set_weights(layerss.get_weights())




  model_cross.fit_generator(
      training_gen(num_training_per_batch,num_few_shots,h_hyper_train_dataset, h_hyper_feature_map, len(h_hyper_train_dataset), SNRdb),
      steps_per_epoch=steps_per_epoch_train,
      epochs=epochs_train,
      validation_data=validation_gen(num_training_per_batch, num_few_shots,h_hyper_train_dataset, h_hyper_feature_map, len(h_hyper_train_dataset), SNRdb),
      validation_steps=1,
      callbacks=[checkpoint],
      verbose=2)

  model_cross.save(saving_path_cross_attention)


  ############################### train task attention ###############################
  net_copy = meta_learning(num_few_shots, feature_dims)
  model_cross_copy = model_cross_attention_CNN(net_copy, num_few_shots, feature_dims)
  model_cross_copy.compile(optimizer='adam', loss='mse',  metrics=[tf.keras.metrics.MeanSquaredError()])
  model_cross_copy = tf.keras.saving.load_model(saving_path_cross_attention)

  net1 = net_copy
  model_cross_Task = model_cross_Task_CNN(net1, num_few_shots, feature_dims)
  model_cross_Task.compile(optimizer='adam', loss='mse',  metrics=[tf.keras.metrics.MeanSquaredError()])
  checkpoint = callbacks.ModelCheckpoint('dual.h5',
                                          verbose=0, save_best_only=True, mode='min', save_weights_only=True)
  layername = ['meta_leaner_1_1','meta_leaner_1_2']
  for i in range(len(layername)):
    for layer in net1.layers:
      if layer.name == layername[i]:
          layer.trainable = False
  layername = ['extractor_1','extractor_2','extractor_3','conv1','conv2','conv3','conv4','conv5','conv6','conv7','channel6_1']
  for i in range(len(layername)):
    for layer in model_cross_Task.layers:
      if layer.name == layername[i]:
          layer.trainable = False

  for i in range(len(layername)):
    for layer in model_cross_Task.layers:
      if layer.name == layername[i]:
        for layerss in model_cross_copy.layers:
          if layerss.name == layername[i]:
            layer.set_weights(layerss.get_weights())




  model_cross_Task.fit_generator(
      training_gen(num_training_per_batch,num_few_shots,h_hyper_train_dataset, h_hyper_feature_map, len(h_hyper_train_dataset),25),
      steps_per_epoch=steps_per_epoch_train,
      epochs=epochs_train,
      validation_data=validation_gen(num_training_per_batch, num_few_shots,h_hyper_train_dataset, h_hyper_feature_map, len(h_hyper_train_dataset), 25),
      validation_steps=1,
      callbacks=[checkpoint],
      verbose=2)

  return model_cross_Task

def test_prediction(model,num_few_shots, h_hyper_test_dataset_various_SNR, h_hyper_testing_feature_map_various_SNR, SNR_list):

  for j in range(num_testing_PDP):
    SNR_record = []
    for z in range(len(SNR_list)):
      acc = 0
      for i in range(num_testing_channel_per_PDP):
        input_zero = np.zeros(shape=(int(2*num_few_shots),K,2))
        H = channel_test[int(j*10000+i+num_few_shots)]
        SNRdb = SNR_list[z]
        h_ls,signal_output, h_true = ofdm_simulate(bitses[i], H, SNRdb)
        h_ls_test = h_ls.reshape(1,h_ls.shape[0],h_ls.shape[1])

        vec_few_shot = h_hyper_test_dataset_various_SNR[z,int(j*10000):int(j*10000+num_few_shots)] # +10000 for D1_4
        vec_few_shot = vec_few_shot.reshape(1,vec_few_shot.shape[0],vec_few_shot.shape[1],vec_few_shot.shape[2])

        vec_feature_map = h_hyper_testing_feature_map_various_SNR[z,int(j*10000):int(j*10000+num_few_shots)] # +10000 for D1_4
        vec_feature_map = vec_feature_map.reshape(1,vec_feature_map.shape[0],vec_feature_map.shape[1],vec_feature_map.shape[2])


        h_pred = model.predict([h_ls_test,vec_few_shot,vec_feature_map], verbose=0)

        H_pred = h_pred[:,:,0] +1j * h_pred[:,:,1]
        #h_pre = np.fft.ifft(H_pred)
        h_pre = H_pred
        h_pred_concat = np.concatenate((np.real(h_pre[0]),np.imag(h_pre[0])))
        H_freq = np.fft.fft(H)
        H_concat = np.concatenate((np.real(H_freq),np.imag(H_freq)))
        mse = ((h_pred_concat - H_concat)**2).mean()
        acc = acc+mse
      acc_mean = acc/num_testing_channel_per_PDP
      SNR_record.append(acc_mean)
    print(SNR_record)

In [ ]:
model_cross_Task = run_experiment(feature_extraction, num_few_shots, feature_dims, h_hyper_train_dataset, h_hyper_feature_map)
test_prediction(model_cross_Task, num_few_shots, h_hyper_test_dataset_various_SNR, h_hyper_testing_feature_map_various_SNR, SNR_list)